In [147]:
import pandas as pd
import string
import re
from string import digits
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
import gensim
import matplotlib.pyplot as plt
import urllib.parse

In [148]:
file1 = pd.read_csv("data/reuters2019-10-26.csv")
file2 = pd.read_csv("data/reuters2019-10-27.csv")
mergedReutersFiles = pd.merge( file1, file2, on=['headline','date','content'],how = 'outer')
mergedReutersFiles.dropna( inplace = True )
null_columns = mergedReutersFiles.columns[mergedReutersFiles.isnull().any()]
print(mergedReutersFiles[mergedReutersFiles.isnull().any(axis=1)][null_columns])
mergedReutersFiles

Empty DataFrame
Columns: []
Index: []


,headline,date,content
0,JS Global pulls Hong Kong listing amid trade g...,"October 26, 2019 / 9:39 AM / Updated 2 minute...",HONG KONG (Reuters) - Household appliances mak...
1,Tech consultations on parts of U.S. trade deal...,"October 26, 2019 / 4:02 AM / Updated 2 hours ago",SHANGHAI (Reuters) - Chinese Vice Premier Liu ...
2,Lagarde will take ECB out of ivory tower: De G...,"October 26, 2019 / 8:35 AM / Updated 2 hours ago",FRANKFURT (Reuters) - Christine Lagarde will e...
3,"U.S., China say they are 'close to finalizing'...","October 25, 2019 / 5:08 AM / Updated 6 hours ago",WASHINGTON (Reuters) - U.S. and Chinese offici...
4,"S&P 500 nears record high on trade progress, u...","October 25, 2019 / 11:53 AM / Updated 8 hours...",NEW YORK (Reuters) - U.S. stocks rose on Frida...
5,Microsoft beats Amazon for Pentagon's $10 bill...,"October 25, 2019 / 11:23 PM / Updated 8 hours...",WASHINGTON (Reuters) - Microsoft Corp. (MSFT.O...
6,UAW turns to Ford after GM workers approve new...,"October 24, 2019 / 9:00 PM / Updated 9 hours ago",DETROIT/WASHINGTON (Reuters) - The United Auto...
7,General Motors to restart Mexican plants after...,"October 26, 2019 / 1:04 AM / Updated 9 hours ago",MEXICO CITY (Reuters) - General Motors Co (GM....
8,Johnson & Johnson CEO testified Baby Powder wa...,"October 22, 2019 / 8:03 PM / Updated 9 hours ago",LOS ANGELES/NEW YORK (Reuters) - Facing off ag...
9,Timeline: UAW members ratify new labor deal wi...,"October 25, 2019 / 10:19 PM / Updated 13 hour...",(Reuters) - The United Auto Workers union said...


In [149]:
 def preprocessing(x):
#     x = x.replace('—', ' ')
    x = x.replace('(Reuters)', ' ')
    x = re.sub('â€™', "'", x)
    x = x.replace ( '[^\x00-\x7f]', 's' )
    
    return x

def processing( dates ):
    dates = dates.split( '/' )
    final_date = dates[0] + '/' + dates[1]
    
    return final_date
mergedReutersFiles['content'] = mergedReutersFiles['content'].apply(preprocessing)
mergedReutersFiles['date'] = mergedReutersFiles['date'].apply(processing)
mergedReutersFiles.index = mergedReutersFiles.index + 1
cols = [ 'content','headline','date' ]
mergedReutersFiles = mergedReutersFiles[cols]
mergedReutersFiles.rename( columns = { 'date' : 'time' }, inplace = True )
mergedReutersFiles

,content,headline,time
1,HONG KONG - Household appliances maker JS Gl...,JS Global pulls Hong Kong listing amid trade g...,"October 26, 2019 / 9:39 AM"
2,SHANGHAI - Chinese Vice Premier Liu He had a...,Tech consultations on parts of U.S. trade deal...,"October 26, 2019 / 4:02 AM"
3,FRANKFURT - Christine Lagarde will ensure Eu...,Lagarde will take ECB out of ivory tower: De G...,"October 26, 2019 / 8:35 AM"
4,WASHINGTON - U.S. and Chinese officials are ...,"U.S., China say they are 'close to finalizing'...","October 25, 2019 / 5:08 AM"
5,NEW YORK - U.S. stocks rose on Friday after ...,"S&P 500 nears record high on trade progress, u...","October 25, 2019 / 11:53 AM"
6,WASHINGTON - Microsoft Corp. (MSFT.O) has wo...,Microsoft beats Amazon for Pentagon's $10 bill...,"October 25, 2019 / 11:23 PM"
7,DETROIT/WASHINGTON - The United Auto Workers...,UAW turns to Ford after GM workers approve new...,"October 24, 2019 / 9:00 PM"
8,MEXICO CITY - General Motors Co (GM.N) will ...,General Motors to restart Mexican plants after...,"October 26, 2019 / 1:04 AM"
9,LOS ANGELES/NEW YORK - Facing off against a ...,Johnson & Johnson CEO testified Baby Powder wa...,"October 22, 2019 / 8:03 PM"
10,- The United Auto Workers union said on Frid...,Timeline: UAW members ratify new labor deal wi...,"October 25, 2019 / 10:19 PM"


In [151]:
mergedReutersFiles.to_csv('data/merged_cleaned_Reuters.csv',encoding='utf-8-sig')


Empty DataFrame
Columns: []
Index: []
